<a href="https://colab.research.google.com/github/leotodisco/QUALITY/blob/main/Dataset_Originale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import library

In [ ]:
%pip install lazypredict
%pip install lightgbm
%pip install seaborn
%pip install matplotlib
%pip install aif360
%pip install 'aif360[Reductions]'
%pip install 'aif360[inFairness]'

## K-Fold Experiments and Models Selection

LazyPredict test (without k-fold)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

data_path = "/content/dataset_originale.csv"
df = pd.read_csv(data_path)
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [00:03<00:00,  9.06it/s]


[LightGBM] [Info] Number of positive: 241, number of negative: 559
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000505 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 446
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.301250 -> initscore=-0.841353
[LightGBM] [Info] Start training from score -0.841353
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.81,0.75,0.75,0.81,0.19
LinearDiscriminantAnalysis,0.81,0.74,0.74,0.80,0.09
LinearSVC,0.81,0.74,0.74,0.80,0.16
RidgeClassifier,0.81,0.74,0.74,0.80,0.05
SGDClassifier,0.75,0.73,0.73,0.76,0.10
LogisticRegression,0.80,0.73,0.73,0.79,0.09
ExtraTreesClassifier,0.80,0.73,0.73,0.79,0.25
GaussianNB,0.71,0.72,0.72,0.73,0.05
LGBMClassifier,0.79,0.72,0.72,0.78,0.17


K-fold (k=10) cross validation, collecting Accuracy and F1-Score

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(270))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
          result.loc[counter]["Fold"] = foldcounter
          result.loc[counter]["Model"] = model[0]
          result.loc[counter]["Accuracy"] = round(model[1][0],3)
          result.loc[counter]["F1-Score"] = round(model[1][3],3)
          counter += 1
          foldcounter += 1

result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [00:04<00:00,  4.79it/s]

[LightGBM] [Info] Number of positive: 267, number of negative: 633
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000332 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 900, number of used features: 58
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.296667 -> initscore=-0.863222
[LightGBM] [Info] Start training from score -0.863222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


100%|██████████| 29/29 [00:05<00:00,  5.69it/s]


Processing Fold 27 ...


...

100%|██████████| 29/29 [00:02<00:00, 10.36it/s]


[LightGBM] [Info] Number of positive: 269, number of negative: 631
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000306 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 451
[LightGBM] [Info] Number of data points in the train set: 900, number of used features: 58
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.298889 -> initscore=-0.852594
[LightGBM] [Info] Start training from score -0.852594
Processing Fold 235 ...


100%|██████████| 29/29 [00:03<00:00,  8.01it/s]


[LightGBM] [Info] Number of positive: 266, number of negative: 634
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 448
[LightGBM] [Info] Number of data points in the train set: 900, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.295556 -> initscore=-0.868553
[LightGBM] [Info] Start training from score -0.868553


,Fold,Model,Accuracy,F1-Score
0,1,SGDClassifier,0.75,0.75
1,2,NuSVC,0.80,0.78
2,3,GaussianNB,0.73,0.73
3,4,NearestCentroid,0.73,0.73
4,5,LGBMClassifier,0.77,0.75
...,...,...,...,...
265,NaN,NaN,NaN,NaN
266,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN
268,NaN,NaN,NaN,NaN


In [ ]:
print(result.groupby('Model', as_index=False)['Accuracy'].mean().sort_values(by='Accuracy', ascending=False))

                            Model Accuracy
17                          NuSVC     0.78
25                            SVC     0.76
21         RandomForestClassifier     0.76
10                 LGBMClassifier     0.76
15             LogisticRegression     0.76
22                RidgeClassifier     0.76
0              AdaBoostClassifier     0.76
14                      LinearSVC     0.75
13     LinearDiscriminantAnalysis     0.75
23              RidgeClassifierCV     0.75
3          CalibratedClassifierCV     0.75
7            ExtraTreesClassifier     0.75
1               BaggingClassifier     0.74
2                     BernoulliNB     0.73
9            KNeighborsClassifier     0.73
24                  SGDClassifier     0.72
19                     Perceptron     0.72
5                 DummyClassifier     0.70
11               LabelPropagation     0.70
12                 LabelSpreading     0.70
18    PassiveAggressiveClassifier     0.69
16                NearestCentroid     0.69
4          

In [ ]:
print(result.groupby('Model', as_index=False)['F1-Score'].mean().sort_values(by='F1-Score', ascending=False))

                            Model F1-Score
17                          NuSVC     0.77
10                 LGBMClassifier     0.75
0              AdaBoostClassifier     0.75
15             LogisticRegression     0.75
14                      LinearSVC     0.74
25                            SVC     0.74
13     LinearDiscriminantAnalysis     0.74
22                RidgeClassifier     0.74
21         RandomForestClassifier     0.74
23              RidgeClassifierCV     0.74
2                     BernoulliNB     0.73
7            ExtraTreesClassifier     0.73
3          CalibratedClassifierCV     0.73
1               BaggingClassifier     0.72
24                  SGDClassifier     0.71
9            KNeighborsClassifier     0.71
19                     Perceptron     0.70
16                NearestCentroid     0.70
11               LabelPropagation     0.69
12                 LabelSpreading     0.69
4          DecisionTreeClassifier     0.68
18    PassiveAggressiveClassifier     0.68
8          

The 5 best results are provided by:

- **NuSVC** (Accuracy: 0.78, F1: 0.77)
- **LGBMClassifier** (Accuracy: 0.76, F1: 0.75)
- **SVC** (Accuracy: 0.76, F1: 0.74)
- **AdaBoostClassifier** (Accuracy: 0.76, F1: 0.75)
- **LogisticRegression** (Accuracy: 0.76, F1: 0.75)

## HyperParameter Optimization

### GridSearchCV - LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

data_path = "/content/dataset_originale.csv"
dataset = pd.read_csv(data_path)
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,random_state =42) #test size 0.2 (80/20) e random state 42

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
logistic_regression_model = LogisticRegression(n_jobs=-1, random_state=0)

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300, 500, 1000],
    'class_weight': [None, 'balanced']
}


precision_scorer = make_scorer(precision_score, zero_division=0)
custom_scoring = {"accuracy": "accuracy", "precision": precision_scorer, "recall": "recall", "f1": "f1"}

grid_logisticRegression = GridSearchCV(logistic_regression_model, param_grid, refit = 'accuracy', verbose = 3, scoring=custom_scoring, cv=10)

categorical_features = ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17', 'Attribute19', 'Attribute20']

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False )
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', one_hot_encoder, categorical_features)
    ],
    remainder='passthrough'  # Lascia le altre colonne inalterate
)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

print("Shape di X_train:", X_train.shape)
print("Shape di X_test:", X_test.shape)

# fitting the model for grid search
grid_logisticRegression.fit(X_train, y_train)

# print best parameter after tuning
print(grid_logisticRegression.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_logisticRegression.best_estimator_)

grid_logisticRegression.score(X_train, y_train)

Streaming output truncated to the last 5000 lines.
[CV 3/10] END C=10, class_weight=balanced, max_iter=100, penalty=l1, solver=newton-cg; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 4/10] END C=10, class_weight=balanced, max_iter=100, penalty=l1, solver=newton-cg; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 5/10] END C=10, class_weight=balanced, max_iter=100, penalty=l1, solver=newton-cg; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 6/10] END C=10, class_weight=balanced, max_iter=100, penalty=l1, solver=newton-cg; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 7/10] END C=10, class_weight=balanced, max_iter=100, penalty=l1, solver=newton-cg; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 8/10] END C=10, class_weight=b

0.77625

## GridSearchCV - NuSVC

In [ ]:
from sklearn.svm import NuSVC

nusvc_model = NuSVC()

param_grid = {
    'nu': [0.1, 0.3, 0.5, 0.7, 0.9],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],  # Applicabile solo se kernel è 'poly'
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
    'coef0': [0.0, 0.1, 0.5, 1.0],  # Applicabile solo se kernel è 'poly' o 'sigmoid'
    'class_weight': [None, 'balanced']
}


precision_scorer = make_scorer(precision_score, zero_division=0)
custom_scoring = {"accuracy": "accuracy", "precision": precision_scorer, "recall": "recall", "f1": "f1"}

grid_svc = GridSearchCV(nusvc_model, param_grid, refit = 'accuracy', verbose = 3, scoring=custom_scoring, cv=10)

# fitting the model for grid search
grid_svc.fit(X_train, y_train)

# print best parameter after tuning
print(grid_svc.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_svc.best_estimator_)

grid_svc.score(X_train, y_train)

Fitting 10 folds for each of 4480 candidates, totalling 44800 fits
[CV 1/10] END class_weight=None, coef0=0.0, degree=2, gamma=scale, kernel=linear, nu=0.1; accuracy: (test=0.450) f1: (test=0.463) precision: (test=0.704) recall: (test=0.345) total time= 8.6min
[CV 2/10] END class_weight=None, coef0=0.0, degree=2, gamma=scale, kernel=linear, nu=0.1; accuracy: (test=0.275) f1: (test=0.256) precision: (test=0.455) recall: (test=0.179) total time= 9.7min
[CV 3/10] END class_weight=None, coef0=0.0, degree=2, gamma=scale, kernel=linear, nu=0.1; accuracy: (test=0.362) f1: (test=0.400) precision: (test=0.586) recall: (test=0.304) total time= 5.9min
[CV 4/10] END class_weight=None, coef0=0.0, degree=2, gamma=scale, kernel=linear, nu=0.1; accuracy: (test=0.412) f1: (test=0.484) precision: (test=0.629) recall: (test=0.393) total time= 6.2min
[CV 5/10] END class_weight=None, coef0=0.0, degree=2, gamma=scale, kernel=linear, nu=0.1; accuracy: (test=0.338) f1: (test=0.329) precision: (test=0.565) rec

## AdaBoostClassifier

In [ ]:
from sklearn.svm import NuSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_model = AdaBoostClassifier()

# Definisci la griglia di iperparametri
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.5, 1, 1.5, 2],
    'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)]
}

precision_scorer = make_scorer(precision_score, zero_division=0)
custom_scoring = {"accuracy": "accuracy", "precision": precision_scorer, "recall": "recall", "f1": "f1"}

grid_ada_boost = GridSearchCV(ada_model, param_grid, refit = 'accuracy', verbose = 3, scoring=custom_scoring, cv=10)

# fitting the model for grid search
grid_ada_boost.fit(X_train, y_train)

# print best parameter after tuning
print(grid_ada_boost.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_ada_boost.best_estimator_)

grid_ada_boost.score(X_train, y_train)

## GridSearchCV - LGBMClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifierCV
import lightgbm as lgb

lgbm_model = lgb.LGBMClassifier()

param_grid = {
    'num_leaves': [50, 100, 200],
    'max_depth': [-1, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 500, 1000],
    'min_child_samples': [20, 50, 100],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.5, 1.0],
    'reg_lambda': [0, 0.5, 1.0]
}


precision_scorer = make_scorer(precision_score, zero_division=0)
custom_scoring = {"accuracy": "accuracy", "precision": precision_scorer, "recall": "recall", "f1": "f1"}
# cambiare ridge a lgbm
grid_ridge = GridSearchCV(lgbm_model, param_grid, refit = 'accuracy', verbose = 3, scoring=custom_scoring, cv=10)

# fitting the model for grid search
grid_ridge.fit(X_train, y_train)

# print best parameter after tuning
print(grid_ridge.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_ridge.best_estimator_)

grid_ridge.score(X_train, y_train)


## GridSearchCV - SVC

In [ ]:
from sklearn.svm import SVC

# Definisci il modello SVC
svc_model = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced']
}

precision_scorer = make_scorer(precision_score, zero_division=0)
custom_scoring = {"accuracy": "accuracy", "precision": precision_scorer, "recall": "recall", "f1": "f1"}

grid_svc = GridSearchCV(svc_model, param_grid, refit = 'accuracy', verbose = 3, scoring=custom_scoring, cv=10)

# fitting the model for grid search
grid_svc.fit(X_train, y_train)

# print best parameter after tuning
print(grid_svc.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_svc.best_estimator_)

grid_svc.score(X_train, y_train)

#RQ1

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

data_path = "/content/dataset_originale.csv"
dataset = pd.read_csv(data_path)
categorical_features = ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17', 'Attribute19', 'Attribute20']

X = dataset.drop('target', axis=1)
y = dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

# Creiamo i nomi delle colonne per le feature trasformate
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
# Creiamo i nomi delle colonne per il DataFrame finale
colonne_numeriche = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11', 'Attribute13', 'Attribute16', 'Attribute18']
feature_names = list(ohe_feature_names) + colonne_numeriche

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

ada_boost_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3), \
                                     learning_rate=1.5, n_estimators=200)

ada_boost_model.fit(X_train, y_train)
print("Shape di X_train:", X_train.shape)
print("Shape di X_test:", X_test.shape)


y_pred = ada_boost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy}")

f1 = f1_score(y_test, y_pred)
print(f"F1 score = {f1}")

#RQ2

In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric

"""
- Sex, privileged: Male, unprivileged: Female   | Attribute9
- Age, privileged: Old, unprivileged: Young     | Attribute13

Attribute 9:  (qualitative)
	      Personal status and sex
	      A91 : male   : divorced/separated
	      A92 : female : divorced/separated/married
        A93 : male   : single
	      A94 : male   : married/widowed
	      A95 : female : single
"""
# Nel dataset originale non si presenta mai il valore A95
sex_features = ['Attribute9_A91','Attribute9_A92','Attribute9_A93','Attribute9_A94'] # attributi protetti

fairness_dataset = pd.DataFrame(X_test.copy(), columns=feature_names)
fairness_dataset['target'] = y_test.to_numpy() # and join the target feature with the others

predictions = fairness_dataset
predictions['target'] = y_pred # but this time the target feature is made by the predictions of our model

# This is the object made of the original dataset
aif_sex_dataset = BinaryLabelDataset( # Base class for all structured datasets with binary labels.
        df=fairness_dataset,
        favorable_label=2.0, # This means that a prediction is biased toward the privileged attribute if its value is 2 (True)
        unfavorable_label=1.0,
        label_names=['target'],
        protected_attribute_names=sex_features,
        privileged_protected_attributes=['Attribute9_A91', 'Attribute9_A93', 'Attribute9_A94'] # here we tell AIF that we want to check for predictions that somehow privilege the attribute "sex_Male"
)

# We do the same thing but with the predictions dataset
aif_sex_pred = BinaryLabelDataset(
        df=predictions,
        favorable_label=2.0, # This means that a prediction is biased toward the privileged attribute if its value is 2 (True)
        unfavorable_label=1.0,
        label_names=['target'],
        protected_attribute_names=sex_features,
        privileged_protected_attributes=['Attribute9_A91', 'Attribute9_A93', 'Attribute9_A94'] # here we tell AIF that we want to check for predictions that somehow privilege the attribute "sex_Male"
)


sex_privileged_group = [{'Attribute9_A91':1.0,
                         'Attribute9_A92':0.0,
                         },
                        {'Attribute9_A93':1.0,
                         'Attribute9_A92':0.0,
                         },
                        {'Attribute9_A94':1.0,
                         'Attribute9_A92':0.0,
                         },
                        ] # The privileged group is made of males

sex_unprivileged_group = [{'Attribute9_A92':1.0,
                         'Attribute9_A91':0.0,
                         'Attribute9_A93':0.0,
                         'Attribute9_A94':0.0,
                         },] # The unprivileged group is made of females

# We provide the ClassificationMetric object with all the information needed:
# aif_sex_dataset - The original test set
# aif_sex_pred - A dataset containing the predictions of the model
# sex_privileged_group - The privileged group
# sex_unprivileged_group - The unprivileged group
fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
                               classified_dataset=aif_sex_pred,
                               unprivileged_groups=sex_unprivileged_group,
                               privileged_groups=sex_privileged_group)

# Values less than 0 indicate that privileged group has higher
# proportion of predicted positive outcomes than unprivileged group.
# Value higher than 0 indicates that unprivileged group has higher proportion
# of predicted positive outcomes than privileged group.
SPD = round(fairness_metrics.statistical_parity_difference(),3)

# Measures the deviation from the equality of opportunity, which means that the same
# proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
EOD = round(fairness_metrics.equal_opportunity_difference(),3)

# Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
# A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
# groups have the same probability of being classified positively.
AOD = round(fairness_metrics.average_odds_difference(),3)

print(f"Statistical Parity Difference (SPD): {SPD}.  The ideal value of this metric is 0") # The ideal value of this metric is 0
print(f"Equal Opportunity Difference (EOD): {EOD}. The ideal value is 0.")
print(f"Average Odds Difference (AOD): {AOD}. The ideal value of this metric is 0.")

In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame and 'y_pred' contains the model predictions
# Ensure 'df' has been defined earlier in your script

"""
- Age, privileged: Old (>= 30), unprivileged: Young (< 30) | Attribute13

Attribute 13: Continuous values for age
"""

# Create a binary column for age
df['Age_Old'] = np.where(df['Attribute13'] >= 30, 1, 0)
df['Age_Young'] = np.where(df['Attribute13'] < 30, 1, 0)

categorical_features = ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17', 'Attribute19', 'Attribute20']

X = df.drop(['target', 'Attribute13'], axis=1)  # Drop original age column
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', one_hot_encoder, categorical_features)
    ],
    remainder='passthrough'  # Leave other columns untouched
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Create column names for the transformed features
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
numeric_features = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11', 'Attribute16', 'Attribute18', 'Age_Old', 'Age_Young']
feature_names = list(ohe_feature_names) + numeric_features

age_features = ['Age_Old', 'Age_Young']  # protected attributes for age

# Create DataFrame for fairness evaluation
fairness_dataset = pd.DataFrame(X_test.copy(), columns=feature_names)
fairness_dataset['target'] = y_test.to_numpy()  # join the target feature with the others

predictions = fairness_dataset.copy()
predictions['target'] = y_pred  # the target feature is made by the predictions of our model

# Create BinaryLabelDataset for the original dataset
aif_age_dataset = BinaryLabelDataset(
        df=fairness_dataset,
        favorable_label=2.0,  # biased towards the privileged attribute if value is 2 (True)
        unfavorable_label=1.0,
        label_names=['target'],
        protected_attribute_names=age_features,
        privileged_protected_attributes=[{'Age_Old': 1.0}]  # privileged: old
)

# Create BinaryLabelDataset for the predictions
aif_age_pred = BinaryLabelDataset(
        df=predictions,
        favorable_label=2.0,
        unfavorable_label=1.0,
        label_names=['target'],
        protected_attribute_names=age_features,
        privileged_protected_attributes=[{'Age_Old': 1.0}]  # privileged: old
)

# Define privileged and unprivileged groups
age_privileged_group = [{'Age_Old': 1.0}]
age_unprivileged_group = [{'Age_Young': 1.0}]

# Compute fairness metrics
fairness_metrics = ClassificationMetric(dataset=aif_age_dataset,
                                        classified_dataset=aif_age_pred,
                                        unprivileged_groups=age_unprivileged_group,
                                        privileged_groups=age_privileged_group)

# Values less than 0 indicate that the privileged group has a higher
# proportion of predicted positive outcomes than the unprivileged group.
# Values higher than 0 indicate that the unprivileged group has a higher
# proportion of predicted positive outcomes than the privileged group.
SPD = round(fairness_metrics.statistical_parity_difference(), 3)

# Measures the deviation from equality of opportunity, meaning that the same
# proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
EOD = round(fairness_metrics.equal_opportunity_difference(), 3)

# Average of the difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
# A value of 0 indicates equality of odds, meaning that samples in both the privileged and unprivileged
# groups have the same probability of being classified positively.
AOD = round(fairness_metrics.average_odds_difference(), 3)

print(f"Statistical Parity Difference (SPD): {SPD}. The ideal value of this metric is 0")  # The ideal value of this metric is 0
print(f"Equal Opportunity Difference (EOD): {EOD}. The ideal value is 0.")
print(f"Average Odds Difference (AOD): {AOD}. The ideal value of this metric is 0.")
